# insert title

## Introduction

paste text here

## Methods & Results

paste description of method here

In [ ]:
# load libraries
library(tidyverse) # loading for analysis
library(RColorBrewer) # loading for colorblind friendly graphs
options(repr.matrix.max.rows = 5)


In [2]:
# load datasets
url = 'https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz'
url2 = 'https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB'
download.file(url, 'players.csv') 
download.file(url2, 'sessions.csv')

players <- read_csv('players.csv') # reading players data
sessions <- read_csv('sessions.csv') |>  # reading session data 
    separate(start_time, into = c('start_date', 'start_time'), sep = ' ') |> # tidying time and date column
    separate(end_time, into = c('end_date', 'end_time'), sep = ' ')

playdata <- merge(players, sessions, by.x = 'hashedEmail', by.y = 'hashedEmail') # combining datasets

set.seed(100) # seed

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Discussion

stuff

## References

if needed